# Parameter Sweeps and Batch Simulation

This notebook demonstrates how to perform parameter sweeps and batch simulations with Pulsim.

## Contents
1. Single Parameter Sweep
2. Multi-Parameter Sweep
3. Parallel Execution
4. Monte Carlo Analysis
5. Optimization Example

In [ ]:
import pulsim
import numpy as np
import matplotlib.pyplot as plt
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm.notebook import tqdm
import time

## 1. Single Parameter Sweep

Let's sweep the duty cycle of a buck converter to characterize its transfer function.

In [ ]:
def create_buck_circuit(duty_cycle):
    """Create a buck converter with specified duty cycle."""
    circuit = pulsim.Circuit(f"Buck D={duty_cycle:.2f}")
    
    # Components
    circuit.add_voltage_source("Vin", "vin", "0", 12.0)
    circuit.add_switch("S_high", "vin", "sw", control="PWM", ron=0.01)
    circuit.add_switch("S_low", "sw", "0", control="PWM_inv", ron=0.01)
    circuit.add_inductor("L1", "sw", "vout", 100e-6)
    circuit.add_capacitor("C1", "vout", "0", 100e-6)
    circuit.add_resistor("R_load", "vout", "0", 5.0)
    
    # PWM sources
    circuit.add_pwm("PWM", frequency=100e3, duty_cycle=duty_cycle)
    circuit.add_pwm("PWM_inv", frequency=100e3, duty_cycle=duty_cycle, 
                    inverted=True, dead_time=100e-9)
    
    return circuit

def simulate_and_measure(duty_cycle):
    """Simulate and return average output voltage."""
    circuit = create_buck_circuit(duty_cycle)
    
    options = pulsim.SimulationOptions(
        stop_time=500e-6,  # 500µs (50 cycles)
        timestep=10e-9
    )
    
    result = pulsim.simulate(circuit, options)
    
    # Average output voltage from last 10 cycles
    ss_mask = result.time > 400e-6
    v_out_avg = np.mean(result.voltages["vout"][ss_mask])
    v_out_ripple = np.max(result.voltages["vout"][ss_mask]) - np.min(result.voltages["vout"][ss_mask])
    
    return v_out_avg, v_out_ripple

In [ ]:
# Sweep duty cycle
duty_cycles = np.linspace(0.1, 0.9, 17)
results = []

print("Running duty cycle sweep...")
for D in tqdm(duty_cycles):
    v_out, v_ripple = simulate_and_measure(D)
    results.append((D, v_out, v_ripple))

# Extract results
D_arr = np.array([r[0] for r in results])
Vout_arr = np.array([r[1] for r in results])
Vripple_arr = np.array([r[2] for r in results])

In [ ]:
# Plot transfer function
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Vout vs D
axes[0].plot(D_arr, Vout_arr, 'bo-', markersize=8, linewidth=2, label='Simulated')
axes[0].plot(D_arr, 12.0 * D_arr, 'r--', linewidth=2, label='Ideal (V_in × D)')
axes[0].set_xlabel('Duty Cycle')
axes[0].set_ylabel('Output Voltage (V)')
axes[0].set_title('Buck Converter Transfer Function')
axes[0].legend()
axes[0].grid(True)

# Ripple vs D
axes[1].plot(D_arr, Vripple_arr * 1e3, 'go-', markersize=8, linewidth=2)
axes[1].set_xlabel('Duty Cycle')
axes[1].set_ylabel('Output Voltage Ripple (mV)')
axes[1].set_title('Output Ripple vs Duty Cycle')
axes[1].grid(True)

plt.tight_layout()
plt.show()

## 2. Multi-Parameter Sweep

Sweep both inductance and load resistance.

In [ ]:
def create_buck_parameterized(L, R_load, D=0.417):
    """Create buck converter with parameterized L and R_load."""
    circuit = pulsim.Circuit(f"Buck L={L*1e6:.0f}µH R={R_load:.0f}Ω")
    
    circuit.add_voltage_source("Vin", "vin", "0", 12.0)
    circuit.add_switch("S_high", "vin", "sw", control="PWM", ron=0.01)
    circuit.add_switch("S_low", "sw", "0", control="PWM_inv", ron=0.01)
    circuit.add_inductor("L1", "sw", "vout", L)
    circuit.add_capacitor("C1", "vout", "0", 100e-6)
    circuit.add_resistor("R_load", "vout", "0", R_load)
    
    circuit.add_pwm("PWM", frequency=100e3, duty_cycle=D)
    circuit.add_pwm("PWM_inv", frequency=100e3, duty_cycle=D, 
                    inverted=True, dead_time=100e-9)
    
    return circuit

def run_sweep_point(params):
    """Worker function for parallel sweep."""
    L, R_load = params
    circuit = create_buck_parameterized(L, R_load)
    
    options = pulsim.SimulationOptions(
        stop_time=500e-6,
        timestep=10e-9
    )
    
    result = pulsim.simulate(circuit, options)
    
    # Measure steady-state
    ss_mask = result.time > 400e-6
    v_out = np.mean(result.voltages["vout"][ss_mask])
    i_L_ripple = np.max(result.currents["L1"][ss_mask]) - np.min(result.currents["L1"][ss_mask])
    
    return L, R_load, v_out, i_L_ripple

In [ ]:
# Define sweep ranges
L_values = np.array([22e-6, 47e-6, 100e-6, 220e-6, 470e-6])  # µH
R_values = np.array([2, 5, 10, 20])  # Ω

# Generate parameter combinations
param_combinations = [(L, R) for L in L_values for R in R_values]

print(f"Running {len(param_combinations)} simulations...")

# Sequential execution (for demonstration)
results_2d = []
for params in tqdm(param_combinations):
    results_2d.append(run_sweep_point(params))

In [ ]:
# Organize results into 2D array
ripple_matrix = np.zeros((len(L_values), len(R_values)))

for L, R, v_out, i_ripple in results_2d:
    i = np.where(L_values == L)[0][0]
    j = np.where(R_values == R)[0][0]
    ripple_matrix[i, j] = i_ripple

# Plot as heatmap
plt.figure(figsize=(10, 8))
plt.imshow(ripple_matrix * 1e3, aspect='auto', origin='lower', cmap='viridis')
plt.colorbar(label='Inductor Current Ripple (mA)')

plt.xticks(range(len(R_values)), [f'{R}Ω' for R in R_values])
plt.yticks(range(len(L_values)), [f'{L*1e6:.0f}µH' for L in L_values])
plt.xlabel('Load Resistance')
plt.ylabel('Inductance')
plt.title('Inductor Current Ripple vs L and R_load')

# Add values as text
for i in range(len(L_values)):
    for j in range(len(R_values)):
        plt.text(j, i, f'{ripple_matrix[i,j]*1e3:.0f}',
                 ha='center', va='center', color='white', fontsize=10)

plt.tight_layout()
plt.show()

## 3. Parallel Execution

Use multiprocessing for faster sweeps.

In [ ]:
# Parallel sweep example
duty_cycles_fine = np.linspace(0.1, 0.9, 33)

def parallel_sweep_point(D):
    """Single sweep point for parallel execution."""
    v_out, v_ripple = simulate_and_measure(D)
    return D, v_out, v_ripple

# Run in parallel
print(f"Running {len(duty_cycles_fine)} simulations in parallel...")
start_time = time.time()

parallel_results = []
with ProcessPoolExecutor(max_workers=8) as executor:
    futures = {executor.submit(parallel_sweep_point, D): D for D in duty_cycles_fine}
    
    for future in tqdm(as_completed(futures), total=len(futures)):
        result = future.result()
        parallel_results.append(result)

elapsed = time.time() - start_time
print(f"Completed in {elapsed:.1f} seconds ({len(duty_cycles_fine)/elapsed:.1f} sims/sec)")

# Sort by duty cycle
parallel_results.sort(key=lambda x: x[0])

In [ ]:
# Plot parallel results
D_fine = [r[0] for r in parallel_results]
Vout_fine = [r[1] for r in parallel_results]

plt.figure(figsize=(10, 6))
plt.plot(D_fine, Vout_fine, 'b.-', markersize=8, linewidth=1, label='Simulated')
plt.plot(D_fine, 12.0 * np.array(D_fine), 'r--', linewidth=2, label='Ideal')
plt.xlabel('Duty Cycle')
plt.ylabel('Output Voltage (V)')
plt.title('Buck Converter Transfer Function (Fine Sweep)')
plt.legend()
plt.grid(True)
plt.show()

## 4. Monte Carlo Analysis

Analyze the effect of component tolerances using Monte Carlo simulation.

In [ ]:
def monte_carlo_simulation(n_samples=100):
    """Run Monte Carlo analysis with component variations."""
    
    # Nominal values
    L_nom = 100e-6  # ±20% tolerance
    C_nom = 100e-6  # ±20% tolerance
    R_nom = 5.0     # ±5% tolerance
    
    results = []
    
    for i in tqdm(range(n_samples)):
        # Apply random variations
        L = L_nom * (1 + 0.20 * np.random.uniform(-1, 1))
        C = C_nom * (1 + 0.20 * np.random.uniform(-1, 1))
        R = R_nom * (1 + 0.05 * np.random.uniform(-1, 1))
        
        # Create circuit with varied components
        circuit = pulsim.Circuit(f"Monte Carlo {i}")
        circuit.add_voltage_source("Vin", "vin", "0", 12.0)
        circuit.add_switch("S_high", "vin", "sw", control="PWM", ron=0.01)
        circuit.add_switch("S_low", "sw", "0", control="PWM_inv", ron=0.01)
        circuit.add_inductor("L1", "sw", "vout", L)
        circuit.add_capacitor("C1", "vout", "0", C)
        circuit.add_resistor("R_load", "vout", "0", R)
        circuit.add_pwm("PWM", frequency=100e3, duty_cycle=0.417)
        circuit.add_pwm("PWM_inv", frequency=100e3, duty_cycle=0.417, 
                        inverted=True, dead_time=100e-9)
        
        # Simulate
        options = pulsim.SimulationOptions(stop_time=500e-6, timestep=10e-9)
        result = pulsim.simulate(circuit, options)
        
        # Measure
        ss_mask = result.time > 400e-6
        v_out = np.mean(result.voltages["vout"][ss_mask])
        v_ripple = np.max(result.voltages["vout"][ss_mask]) - np.min(result.voltages["vout"][ss_mask])
        
        results.append((L, C, R, v_out, v_ripple))
    
    return results

print("Running Monte Carlo analysis...")
mc_results = monte_carlo_simulation(100)

In [ ]:
# Extract Monte Carlo data
v_out_mc = np.array([r[3] for r in mc_results])
v_ripple_mc = np.array([r[4] for r in mc_results])

# Statistics
print("Monte Carlo Results (n=100):")
print(f"\nOutput Voltage:")
print(f"  Mean: {np.mean(v_out_mc):.4f} V")
print(f"  Std:  {np.std(v_out_mc):.4f} V")
print(f"  Min:  {np.min(v_out_mc):.4f} V")
print(f"  Max:  {np.max(v_out_mc):.4f} V")

print(f"\nOutput Ripple:")
print(f"  Mean: {np.mean(v_ripple_mc)*1e3:.2f} mV")
print(f"  Std:  {np.std(v_ripple_mc)*1e3:.2f} mV")
print(f"  Min:  {np.min(v_ripple_mc)*1e3:.2f} mV")
print(f"  Max:  {np.max(v_ripple_mc)*1e3:.2f} mV")

In [ ]:
# Plot Monte Carlo distributions
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Output voltage distribution
axes[0].hist(v_out_mc, bins=20, edgecolor='black', alpha=0.7)
axes[0].axvline(x=5.0, color='r', linestyle='--', linewidth=2, label='Target: 5.0V')
axes[0].axvline(x=np.mean(v_out_mc), color='g', linestyle='-', linewidth=2, 
                label=f'Mean: {np.mean(v_out_mc):.3f}V')
axes[0].set_xlabel('Output Voltage (V)')
axes[0].set_ylabel('Count')
axes[0].set_title('Output Voltage Distribution')
axes[0].legend()

# Ripple distribution
axes[1].hist(v_ripple_mc * 1e3, bins=20, edgecolor='black', alpha=0.7, color='orange')
axes[1].axvline(x=np.mean(v_ripple_mc)*1e3, color='g', linestyle='-', linewidth=2,
                label=f'Mean: {np.mean(v_ripple_mc)*1e3:.1f}mV')
axes[1].set_xlabel('Output Ripple (mV)')
axes[1].set_ylabel('Count')
axes[1].set_title('Output Ripple Distribution')
axes[1].legend()

plt.tight_layout()
plt.show()

## 5. Optimization Example

Find the optimal inductance to minimize ripple while meeting a size constraint.

In [ ]:
from scipy.optimize import minimize_scalar

def ripple_objective(L_uH):
    """Objective function: return ripple for given inductance."""
    L = L_uH * 1e-6
    
    circuit = create_buck_parameterized(L, R_load=5.0, D=0.417)
    options = pulsim.SimulationOptions(stop_time=300e-6, timestep=10e-9)
    result = pulsim.simulate(circuit, options)
    
    ss_mask = result.time > 200e-6
    v_ripple = np.max(result.voltages["vout"][ss_mask]) - np.min(result.voltages["vout"][ss_mask])
    
    return v_ripple * 1e3  # Return in mV

# Find optimal L (between 10µH and 500µH)
print("Optimizing inductance...")
result_opt = minimize_scalar(ripple_objective, bounds=(10, 500), method='bounded')

print(f"\nOptimal inductance: {result_opt.x:.1f} µH")
print(f"Minimum ripple: {result_opt.fun:.2f} mV")

In [ ]:
# Verify by plotting ripple vs inductance
L_test = np.logspace(1, 2.7, 20)  # 10 to 500 µH
ripple_test = [ripple_objective(L) for L in tqdm(L_test)]

plt.figure(figsize=(10, 6))
plt.semilogx(L_test, ripple_test, 'bo-', markersize=8, linewidth=2)
plt.axvline(x=result_opt.x, color='r', linestyle='--', linewidth=2, 
            label=f'Optimal: {result_opt.x:.0f}µH')
plt.xlabel('Inductance (µH)')
plt.ylabel('Output Voltage Ripple (mV)')
plt.title('Ripple vs Inductance Optimization')
plt.legend()
plt.grid(True, which='both', alpha=0.5)
plt.show()

## Summary

This notebook covered:
- Single parameter sweeps (duty cycle)
- Multi-parameter sweeps (L and R_load)
- Parallel execution with ProcessPoolExecutor
- Monte Carlo analysis for component tolerances
- Parameter optimization using scipy.optimize

Key techniques:
- Use `ProcessPoolExecutor` for parallel sweeps
- Monte Carlo with uniform distributions for tolerance analysis
- `scipy.optimize` for design optimization
- Organize results into matrices for 2D visualization